In [34]:
#import any library dependencies
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

In [56]:
#read in the data
dataset = pd.read_csv("country_pred_delta_forest.csv")
df = dataset.drop(columns=["Area","Unnamed: 0"])
#df = df[df["Country.Code"] != "NRU"]
#df = df[df["Country.Code"] != "QAT"]
#df = df[df['Country.Code'] != 'SMR']
#create a lag by country
df["delta_forest_area_lag_1"] = df.groupby(["Country.Code"])["delta_forest_area"].shift(1)

In [57]:
#for_naive.head()
#countries["USA"]
#df.head()
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5252 entries, 0 to 5251
Data columns (total 116 columns):
Country.Code                        object
yr                                  int64
countryname                         object
pct_forest                          float64
gdp                                 float64
pop                                 float64
forest_area                         float64
x_6796_722511                       float64
x_6796_723011                       float64
x_6796_723111                       float64
x_6796_724311                       float64
x_6796_724411                       float64
x_6796_7245                         float64
x_6796_7246                         float64
x_6797_722511                       float64
x_6797_723011                       float64
x_6797_723111                       float64
x_6797_724311                       float64
x_6797_724411                       float64
x_6797_7245                         float64
x_6797_7246     

# Create a dataset with internal splits by country

In [58]:
#split dataset by country
def to_country(data, cntry_ix):
    countries = dict()
    #get unique country codes
    country = np.unique(data[ : , cntry_ix])
    #group by country
    for c in country:
        select = data[:, cntry_ix] == c
        countries[c] = data[select, :]
    return countries

In [59]:
values = df.values
countries = to_country(values,0)
print('Total countries: %d' % len(countries))

Total countries: 202


# Split data into train and test

Train is years 1991-2011 and test is 2012-2016

In [60]:
#split into train and test sets
def split_train_test(countries, row_in_country):
    train, test = list(), list()
    #First 21 years for train
    cut_point = 2011
    #list out countries
    for x, rows in countries.items():
        #split by position
        train_rows = rows[rows[:,row_in_country] <= cut_point, :]
        test_rows = rows[rows[:,row_in_country]> cut_point, :]
        if len(train_rows) == 0 or len(test_rows) == 0:
            print("Dropping country=%s: train=%s, test=%s" % (x,train_rows.shape,test_rows.shape))
            continue
        #sort with country id, position, year, targets
        indices = [0,1,95,115]
        train.append(train_rows[: ,indices])
        test.append(test_rows[:, indices])
    return train, test

train, test = split_train_test(countries,1)

In [61]:
test[1]

array([['AFG', 2012, 0.0, 0.0],
       ['AFG', 2013, 0.0, 0.0],
       ['AFG', 2014, 0.0, 0.0],
       ['AFG', 2015, 0.0, 0.0],
       ['AFG', 2016, 0.0, 0.0]], dtype=object)

# Persistance [Naive] Model

In [62]:
def persistance_model(x):
    return x

In [63]:
test_y = list()
predictions = list()
for x in test:
    for row in x:
        true = row[3]
        yhat = persistance_model(row[2])
        predictions.append(yhat)
        test_y.append(true)
test_mse = mean_squared_error(test_y,predictions)
test_mae = mean_absolute_error(test_y,predictions)
test_rmse = np.sqrt(test_mse)

In [64]:
print("Test MSE: %f" % test_mse)
print("Test MAE: %f" % test_mae)
print("Test RMSE: %f" % test_rmse)

Test MSE: 0.000002
Test MAE: 0.000228
Test RMSE: 0.001323


# Boruta Feature Selection

In [65]:
#create some dummy features
df = pd.get_dummies(df, columns=["IncomeGroup.x","Sub.Regions.x"])

#df.info(verbose=True)
df = df.fillna(0)

In [66]:
#split our target/features
X = df[['yr','gdp','pop','x_6796_722511','x_6796_723011','x_6796_723111','x_6796_724311',
        'x_6796_724411','x_6796_7245','x_6796_7246','x_6797_722511','x_6797_723011','x_6797_723111','x_6797_724311',
        'x_6797_724411','x_6797_7245','x_6797_7246','x_6798_722511','x_6798_723011','x_6798_723111','x_6798_724311',
        'x_6798_724411','x_6798_7245','x_6798_7246','x_6970_5008','x_6971_5008','x_6972_5008','x_6976_5008',
        'x_6977_5008','x_6978_5008','x_6979_5008','x_6980_5008','x_6981_5008','x_6983_5008','x_6600_5110',
        'x_6601_5110','x_6602_5110','x_6610_5110','x_6620_5110','x_6621_5110','x_6650_5110','x_6655_5110',
        'x_6663_5110','x_6670_5110','x_6716_5110','x_6717_5110','x_2071_5910','x_236_5910','x_237_5910','x_257_5910',
        'x_258_5910','x_1601_5516','x_1602_5516','x_1603_5516','x_1604_5516','x_1623_5516','x_1626_5516',
        'x_1633_5516','x_1634_5516','Commodity.driven.deforestation.x','Forestry.x',
        'Wildfire.x','Shifting.agriculture.x','datagroup','forestgroup','gdpgroup','popgroup','lag_gdp','lag_pop',
        'lag_x_6798_7245','lag_x_6798_7246','lag_x_6610_5110','lag_x_6716_5110','lag_x_6717_5110','lag_x_2071_5910',
        'lag_x_236_5910','lag_x_237_5910','lag_x_257_5910','lag_x_258_5910','lag_x_1601_5516','lag_x_1602_5516',
        'lag_x_1603_5516','lag_x_1604_5516','lag_x_1623_5516','lag_x_1626_5516','lag_x_1633_5516','lag_x_1634_5516',
        'delta_gdp','delta_pop','delta_x_257_5910','delta_x_236_5910','delta_x_2071_5910','delta_x_258_5910',
        'delta_x_237_5910','delta_x_1601_5516','delta_x_1604_5516','delta_x_1623_5516','delta_x_1626_5516',
        'delta_x_1633_5516','delta_x_1602_5516','delta_x_1603_5516','delta_x_1634_5516','delta_x_6798_7246',
        'delta_x_6798_7245','delta_x_6610_5110','delta_x_6716_5110','delta_x_6717_5110','IncomeGroup.x_High income',
        'IncomeGroup.x_Low income','IncomeGroup.x_Lower middle income','IncomeGroup.x_Upper middle income',
        'Sub.Regions.x_Australia and New Zealand','Sub.Regions.x_Central Asia','Sub.Regions.x_Eastern Asia',
        'Sub.Regions.x_Eastern Europe','Sub.Regions.x_Latin America and the Caribbean','Sub.Regions.x_Melanesia',
        'Sub.Regions.x_Micronesia','Sub.Regions.x_Northern Africa','Sub.Regions.x_Northern America',
        'Sub.Regions.x_Northern Europe','Sub.Regions.x_Polynesia','Sub.Regions.x_South-eastern Asia',
        'Sub.Regions.x_Southern Asia','Sub.Regions.x_Southern Europe','Sub.Regions.x_Sub-Saharan Africa',
        'Sub.Regions.x_Western Asia','Sub.Regions.x_Western Europe']]
y = df['delta_forest_area']


In [67]:
###initialize Boruta
forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 5
)
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)
### fit Boruta (it accepts np.array, not pd.DataFrame)
boruta.fit(np.array(X), np.array(y))
### print results
green_area = X.columns[boruta.support_].to_list()
blue_area = X.columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

features in the green area: ['yr', 'gdp', 'pop', 'x_6971_5008', 'x_6972_5008', 'x_6976_5008', 'x_6977_5008', 'x_6978_5008', 'x_6979_5008', 'x_6981_5008', 'x_6983_5008', 'x_6620_5110', 'x_6650_5110', 'x_6655_5110', 'x_6663_5110', 'x_6670_5110', 'x_6716_5110', 'x_6717_5110', 'x_1603_5516', 'x_1633_5516', 'x_1634_5516', 'forestgroup', 'lag_gdp', 'lag_pop', 'lag_x_6716_5110', 'lag_x_6717_5110', 'lag_x_1603_5516', 'lag_x_1604_5516', 'lag_x_1623_5516', 'lag_x_1633_5516', 'lag_x_1634_5516', 'delta_pop', 'delta_x_6716_5110', 'delta_x_6717_5110', 'IncomeGroup.x_Lower middle income', 'Sub.Regions.x_Northern Africa', 'Sub.Regions.x_Polynesia', 'Sub.Regions.x_Sub-Saharan Africa']
features in the blue area: ['x_6970_5008', 'x_6600_5110', 'x_6601_5110', 'x_6602_5110', 'x_6610_5110', 'x_1604_5516', 'lag_x_6610_5110', 'lag_x_257_5910', 'lag_x_1601_5516', 'IncomeGroup.x_High income', 'Sub.Regions.x_Northern Europe']


In [ ]:
# convert the test dataset in chunks to [chunk][variable][time] format
def prepare_test_forecasts(test_chunks):
    predictions = list()
    # enumerate chunks to forecast
    for rows in test_chunks:
    # enumerate targets for chunk
    chunk_predictions = list()
    for j in range(3, rows.shape[1]):
        yhat = rows[:, j]
        chunk_predictions.append(yhat)
        chunk_predictions = array(chunk_predictions)
        predictions.append(chunk_predictions)
    return np.array(predictions)

# calculate the error between an actual and predicted value
def calculate_error(actual, predicted):
    # give the full actual value if predicted is nan
    if isnan(predicted):
        return abs(actual)
    # calculate abs difference
    return abs(actual - predicted)